# Hybrid Search and Retrieval with OpenSearch and Ollama

This notebook demonstrates how to:
1. Set up hybrid search with OpenSearch (combining text and semantic search)
2. Generate embeddings for search queries
3. Perform hybrid search to retrieve relevant document chunks
4. Use Ollama to generate responses based on the retrieved context

First, let's import the necessary dependencies and set up our environment.


In [1]:
# Import necessary libraries
import json
import sys
from typing import Dict, Any, List, Optional, Iterable

import numpy as np
import ollama
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer

# Set up Python path to access project modules
sys.path.insert(0, "..")

%load_ext autoreload
%autoreload 2

EMBEDDING_MODEL_PATH = "sentence-transformers/all-mpnet-base-v2"  # 
ASSYMETRIC_EMBEDDING = False  # Flag for asymmetric embedding
EMBEDDING_DIMENSION = 768  # Embedding model settings
TEXT_CHUNK_SIZE = 300  # Maximum number of characters in each text chunk for
OLLAMA_MODEL_NAME = ("llama3.2:1b") # Name of the model used in Ollama for chat functionality

# Logging
LOG_FILE_PATH = "logs/app.log"  # File path for the application log file

# OpenSearch settings
OPENSEARCH_HOST = "localhost"  # Hostname for the OpenSearch instance
OPENSEARCH_PORT = 9200  # Port number for OpenSearch
OPENSEARCH_INDEX = "documents"  # Index name for storing documents in OpenSearch
## Use http://localhost:9200
# opensearch is runnibg on your local machine instead of a remote server
# you have already started the opensearch container using docker 


/Users/parulpandey/Library/CloudStorage/OneDrive-Personal/Ext Github Repos/RAG_UI_fresh/.venv/lib/python3.13/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Embedding settings
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # Model for generating embeddings
EMBEDDING_DIMENSION = 384  # Embedding dimension for the model
ASSYMETRIC_EMBEDDING = False  # Whether to use asymmetric embeddings


## 1. Connect to OpenSearch and Set Up Hybrid Search

First, we'll connect to our OpenSearch instance and define the hybrid search function that combines:
- Text-based search (BM25)
- Vector-based semantic search (KNN)

The hybrid search will use a pipeline that normalizes and combines scores from both search methods.


In [3]:
# Initialize OpenSearch client
# cleint talks to your host where opensearch is running. Gives tasks like indexing, searching, updating, deleting documents to the host. 
# An OpenSearch client is the official library (in Python, Java, JS, etc.) that wraps OpenSearch’s REST API, so your app can connect, 
# query, and manage the cluster more easily.

client = OpenSearch(
    hosts=[{"host": OPENSEARCH_HOST, "port": OPENSEARCH_PORT}],
    http_compress=True,
    timeout=30,
    max_retries=3,
    retry_on_timeout=True,
)

# Check connection
try:
    info = client.info()
    print(f"Successfully connected to OpenSearch {info['version']['number']}")
except Exception as e:
    print(f"Failed to connect to OpenSearch: {e}")
    print("Make sure OpenSearch is running on localhost:9200")
    raise


Successfully connected to OpenSearch 2.11.0


In [4]:
# Verify pipeline exists
from opensearchpy.exceptions import NotFoundError
pipeline_name = "nlp-search-pipeline"

try:
    result = client.transport.perform_request(
        "GET",
        f"/_search/pipeline/{pipeline_name}"
    )
    print(f"\n✅ Search pipeline '{pipeline_name}' exists.")
    
except NotFoundError:
    print(f"\n⚠️ Search pipeline '{pipeline_name}' does NOT exist.")
    print("This is required for hybrid search. Please run the prerequisites notebook.")
except Exception as e:
    print(f"\n🚨 Error: {e}")


✅ Search pipeline 'nlp-search-pipeline' exists.


## How does open search combing query text and query embedding results 
- Since the range of bm25/knn is in different ranges they are normalized first 
normalized_bm25 = (bm25_score - min_bm25) / (max_bm25 - min_bm25)
normalized_knn  = (knn_score - min_knn) / (max_knn - min_knn)
final_score = w_text * normalized_bm25 + w_knn * normalized_knn

In [5]:
# This function performs a hybrid search for the query text using both text-based and vector-based search methods
# Text-based search is effective for exact matches (BM25) and keyword relevance, while vector-based search captures semantic meaning (knn) 
# and context.
# takes as input both query text and its embedding vector

def hybrid_search(query_text: str, query_embedding: List[float], top_k: int = 5) -> List[Dict[str, Any]]:
    """
    Performs hybrid search combining text-based and vector-based queries.
    
    Args:
        query_text (str): The text query for BM25 search
        query_embedding (List[float]): The vector embedding for KNN search
        top_k (int): Number of results to return
        
    Returns:
        List[Dict[str, Any]]: The search results
    """
    query_body = {
        "_source": {"exclude": ["embedding"]},  # Exclude embeddings from results
        "query": {
            "hybrid": {
                "queries": [
                    {"match": {"text": {"query": query_text}}},  # Text-based search
                    {
                        "knn": {
                            "embedding": {
                                "vector": query_embedding,
                                "k": top_k,
                            }
                        }
                    },  # Vector-based search
                ]
            }
        },
        "size": top_k,
    }
    
    print("\nExecuting hybrid search query...")
    try:
        # Try with search pipeline parameter (for newer OpenSearch versions)
        response = client.search(
            index=OPENSEARCH_INDEX,
            body=query_body,
            params={"search_pipeline": "nlp-search-pipeline"} # Uses the pipeline for score normalization
        )
    except TypeError:
        # Fall back to without pipeline parameter for older versions
        print("Warning: OpenSearch client doesn't support search_pipeline parameter, using raw query")
        response = client.search(
            index=OPENSEARCH_INDEX,
            body=query_body
        )
    
    return response["hits"]["hits"]

### rescore query example

In [6]:

# Combines k-NN search with a text-based rescore query to refine results
# The k-NN search retrieves the top 100 nearest neighbors based on the embedding vector
# The rescore query then re-evaluates the top 50 results using a text match
# The final ranking is a weighted combination of the original k-NN scores and the rescore text match scores

## rescore_query_weight tells OpenSearch how much to trust the rescore query relative to the original one.

{
  "query": {
    "knn": {
      "embedding": {
        "vector": [0.1, 0.4, 0.6],
        "k": 100
      }
    }
  },
  "rescore": {
    "window_size": 50,
    "query": {
      "rescore_query": { "match": { "text": "quantum computers" } },
      "query_weight": 0.3,
      "rescore_query_weight": 0.7
    }
  }
}

{'query': {'knn': {'embedding': {'vector': [0.1, 0.4, 0.6], 'k': 100}}},
 'rescore': {'window_size': 50,
  'query': {'rescore_query': {'match': {'text': 'quantum computers'}},
   'query_weight': 0.3,
   'rescore_query_weight': 0.7}}}

## 2. Process Query and Perform Search

Now we'll demonstrate how to:
1. Process a search query
2. Generate its embedding
3. Perform hybrid search to get relevant document chunks

Let's try a sample query to test our search functionality.


In [7]:
def get_embedding_model():
    """
    Loads and returns the sentence transformer embedding model.
    
    Returns:
        SentenceTransformer: The loaded embedding model.
    """
    print(f"Loading embedding model: {EMBEDDING_MODEL_NAME}")
    model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    return model


def generate_embeddings(texts: List[str]):
    """
    Generates embeddings for a list of text chunks.
    
    Args:
        texts (List[str]): List of text chunks to embed.
        
    Returns:
        List[numpy.ndarray]: List of embedding vectors.
    """
    model = get_embedding_model()
    
    # If using asymmetric embeddings, prefix each text with "passage: "
    if ASSYMETRIC_EMBEDDING:
        texts = [f"passage: {text}" for text in texts]
        
    # Generate embeddings
    embeddings = model.encode(texts)
    return embeddings

In [8]:
# Sample query
#query = "What is the average rate of ice loss"
query = "What is an ice creame"
print(f"Query: '{query}'")

# Generate query embedding
print("\nGenerating embedding for query...")
embeddings = generate_embeddings([query])
query_embedding = embeddings[0].tolist()
print(f"Generated embedding with dimension: {len(query_embedding)}")

# Set number of results to retrieve
top_k = 3
print(f"\nRetrieving top {top_k} documents...")

# Perform hybrid search
results = hybrid_search(query, query_embedding, top_k=top_k)

# Display results
print(f"\nSearch results for query: '{query}'\n")
for i, hit in enumerate(results, 1):
    print(f"Result {i} (Score: {hit['_score']:.3f}):")
    print(f"Text: {hit['_source']['text'][:200]}...")  # Showing truncated text
    print(f"Document: {hit['_source']['document_name']}\n")


Query: 'What is an ice creame'

Generating embedding for query...
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
Generated embedding with dimension: 384

Retrieving top 3 documents...

Executing hybrid search query...

Search results for query: 'What is an ice creame'

Result 1 (Score: 0.700):
Text: ). These ranges are derived from CMIP5 climate projections in combination with process-based models and literature assessment of glacier and ice sheet contributions (see Figure SPM.9, Table SPM.2). {1...
Document: climate

Result 2 (Score: 0.617):
Text: km2 per decade), and very likely in the range 9.4 to 13.6% per decade (range of 0.73 to 1.07 million km2 per decade) for the summer sea ice minimum (perennial sea ice). The average decrease in decadal...
Document: climate

Result 3 (Score: 0.300):
Text: the surface to the deep ocean and affect ocean circulation. {11.3, 12.4} It is very likely that the Arctic sea ice cover will continue to shrink and thin and that Northern He

## 3. Generate Response with Ollama

Finally, we'll use Ollama to generate a response based on the retrieved context. We'll:
1. Format the context and query into a prompt
2. Stream the response from Ollama
3. Display the generated response

Make sure you have Ollama running locally with the specified model pulled.


In [9]:
# Define a function to generate responses with Ollama
def generate_response_with_ollama(query: str, results: List[Dict], model_name: str = OLLAMA_MODEL_NAME):
    """
    Generates a response using Ollama based on search results.
    
    Args:
        query (str): The user's question
        results (List[Dict]): The search results from OpenSearch
        model_name (str): The Ollama model to use
        
    Returns:
        tuple: A tuple containing (prompt, model_name)
    """
    # Format context from search results
    context = ""
    for i, result in enumerate(results):
        context += f"Document {i + 1}:\n{result['_source']['text']}\n\n"

    # Create prompt template
    prompt = f"""You are a helpful AI assistant. Use the following context to answer the question.
If you cannot find the answer in the context, say so.

Context:
{context}

Question: {query}

Answer: """

    return prompt, model_name



In [10]:
# Ensure model is pulled
print(f"Ensuring Ollama model {OLLAMA_MODEL_NAME} is available...")
try:
    ollama.pull(OLLAMA_MODEL_NAME)
    print(f"Model {OLLAMA_MODEL_NAME} is ready.")
except ollama.ResponseError as e:
    print(f"Error pulling model: {e.error}")
    print("You might need to install the model manually with: ollama pull " + OLLAMA_MODEL_NAME)

# Get prompt and model
prompt, model_name = generate_response_with_ollama(query, results)
print('\n\n\n')
print(prompt)
print(f"\nUsing model: {model_name}")

Ensuring Ollama model llama3.2:1b is available...
Model llama3.2:1b is ready.




You are a helpful AI assistant. Use the following context to answer the question.
If you cannot find the answer in the context, say so.

Context:
Document 1:
). These ranges are derived from CMIP5 climate projections in combination with process-based models and literature assessment of glacier and ice sheet contributions (see Figure SPM.9, Table SPM.2). {13.5} • In the RCP projections, thermal expansion accounts for 30 to 55% of 21st century global mean sea level rise, and glaciers for 15 to 35%. The increase in surface melting of the Greenland ice sheet will exceed the increase in snowfall, leading to a positive contribution from changes in surface mass balance to future sea level ( high confidence ). While surface melt - ing will remain small, an increase in snowfall on the Antarctic ice sheet is expected ( medium confidence ), resulting in a negative contribution to future sea level from changes in sur

In [11]:
# Print prompt length
print(f"\nPrompt created with {len(prompt)} characters")
print("First 200 characters of prompt:")
print(prompt[:200] + "...")



Prompt created with 9245 characters
First 200 characters of prompt:
You are a helpful AI assistant. Use the following context to answer the question.
If you cannot find the answer in the context, say so.

Context:
Document 1:
). These ranges are derived from CMIP5 cli...


In [12]:
# Give promt as input to ollama generate function

# Generate streaming response in Ollama: Setting stream=True when calling ollama.generate(...) 
# tells the Ollama client to return an iterator/stream of partial response chunks instead of waiting for the full answer. 

print("\nGenerating response with Ollama...")
response = ""
print("\nResponse:")
for chunk in ollama.generate(model=model_name, prompt=prompt, stream=True):
    piece = chunk['response']
    print(piece, end='', flush=True)
    response += piece

print("\n\nResponse generation complete!")
print(f"Generated {len(response)} characters")


Generating response with Ollama...

Response:
I'm not able to provide a specific definition for "ice cream" as it can refer to different things depending on the context. However, I can provide some possible answers based on common understandings of ice cream:

1. In its most basic sense, ice cream is a frozen dessert made from cream, sugar, and flavorings, typically stored in an ice box or freezer.
2. In a broader cultural context, ice cream can refer to a sweet treat that originated in the Middle East, where it was first mentioned in ancient times.
3. In some regions, particularly in North America, "ice cream" might also be used as a slang term for marijuana.

If you could provide more context or clarify which aspect of "ice cream" you are referring to, I would be more than happy to try and provide a more accurate answer.

Response generation complete!
Generated 788 characters


## Conclusion

Congratulations! You've successfully:
1. Connected to OpenSearch and verified the hybrid search pipeline
2. Generated embeddings for a search query
3. Performed hybrid search combining BM25 and semantic search
4. Generated a response using Ollama based on the retrieved documents

You can experiment with different:
- Search queries
- Hybrid search parameters and weights
- Ollama models and prompts
- Result formatting and processing

This completes the basic RAG (Retrieval Augmented Generation) pipeline that combines the power of OpenSearch for hybrid retrieval with the generation capabilities of LLMs through Ollama.